In [1]:
# pip install langchain duckduckgo-search
# pip install langchain-google-community
# pip install pip-system-certs
# pip install -qU duckduckgo-search langchain-community
import os
cwd_path = os.getcwd()
import warnings
warnings.filterwarnings('ignore')

In [2]:
import streamlit as st
from openai import AzureOpenAI
from langchain.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI
from langchain import tools
from langchain_core.tools import tool, Tool
import langchain_community
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import AIMessage
from langchain.agents import AgentExecutor, create_tool_calling_agent, initialize_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import requests
# from langchain_community.document_loaders import WebBaseLoader, OnlinePDFLoader
# from langchain_community.tools import DuckDuckGoSearchRun
from googlesearch import search

In [3]:
ENDPOINT = os.environ['OPENAI_ENDPOINT']
API_KEY = os.environ['OPENAI_API_KEY']
AZURE_DEPLOYMENT = os.environ['AZURE_DEPLOYMENT']
AZURE_OPENAI_VERSION = os.environ['AZURE_OPENAI_VERSION']

llm = AzureChatOpenAI(
    azure_endpoint=ENDPOINT,
    openai_api_key=API_KEY,
    azure_deployment=AZURE_DEPLOYMENT,
    openai_api_version=AZURE_OPENAI_VERSION
)

In [8]:
@tool
def SearchWebForPdf(query):
    """ Search for pdf dataset in the web. Return list of urls of pdf documents. The input
    to the tool will be a query text to perform the search of the datasheet. For example, if
    query is 'lm741 datasheet pdf' then you need to return all the links of the search results
    in a format of list. For example, if there are 3 search results of Link1, Link2 and Link3, 
    then this function should return [Link1, Link2, Link3]
    """ 
    result = search(query, tld="co.in", num=10, stop=10, pause=2)
    search_list =[]
    for j in result:
        search_list.append(j)
    return search_list

@tool
def DownloadFileTool(search_list):
    """Downloads top pdf files from URLs and saves it to a specified path. The input
    to the tool will be a list of links. All the links will be checked one by one if the link 
    contains pdf file or not. If the link type is pdf, then it will download the pdf
    file at a specified location in local disk"""
    
    failed_file = 0
    file_number = 1
    for url in search_list:    
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()  # Raise an exception for bad status codes
            
            # Check if the content type is PDF
            if 'application/pdf' not in response.headers.get('content-type', '').lower():
                print("Error: URL does not point to a PDF file.")
                failed_file+=1
            else:
                dwnld_path = cwd_path+"\Datasheet_Folder"
                if not os.path.exists(dwnld_path):
                    os.makedirs(dwnld_path)
                file_name = dwnld_path+"\datasheet_pdf_"+str(file_number) + ".pdf"
                with open(file_name, 'wb') as pdf_file:
                    for chunk in response.iter_content(chunk_size=8192):
                        pdf_file.write(chunk)
                print(f"File downloaded successfully")
                file_number+=1
        except requests.exceptions.RequestException as e:
            print(f"Error downloading file: {e}")
            failed_file+=1
    if(failed_file==len(search_list)):
        return ("No pdf file is found downloadable")
    else:
        return ("pdf file(s)) saved successfully")

In [9]:
tools = [SearchWebForPdf, DownloadFileTool]
system_message = """
You are a web search agent. Your job is to search and download datasheets in pdf for a specified
components. You have two helper functions 'SearchWebForPdf' and 'DownloadFileTool'. 'SearchWebForPdf'
will help you to get the all list of urls related to the human query. 'DownloadFileTool' will help
you to download all valid pdfs from the list one by one and store in local disk in a specified location
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        # MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])

# initializing the agent
# agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# agent = create_tool_calling_agent(model, tools, prompt)
agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


human_query = "can you do google search and download datasheet of LCO 7A chip?"
# Input with tool calls
messages = [
    ("user", human_query)
]

result = agent_executor.invoke({"input": messages})
# result = agent.run(human_query)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `SearchWebForPdf` with `{'query': 'LCO 7A chip datasheet pdf'}`


['https://www.intel.com/content/dam/www/public/us/en/documents/datasheets/7-series-chipset-pch-datasheet.pdf', 'https://www.ti.com/lit/gpn/SN74ALVC125', 'https://www.st.com/resource/en/datasheet/l78.pdf', 'https://soldered.com/productdata/2015/09/Soldered_LD3361BS_datasheet.pdf?srsltid=AfmBOoo6T8_8RKeBlCf5kDR7RmTL9CKU1ypQl5TjRoJjnMJ8ulsTrirV', 'https://www.power.com/sites/default/files/documents/lytswitch-7_family_datasheet.pdf', 'https://www.we-online.com/components/products/datasheet/157142V12703.pdf', 'https://www.analog.com/media/en/technical-documentation/data-sheets/op07.pdf', 'https://www.st.com/resource/en/datasheet/l4931.pdf', 'https://www.ti.com/lit/gpn/CY74FCT2827T', 'https://www.microchip.com/mymicrochip/filehandler.aspx?ddocname=en011547']
Invoking: `DownloadFileTool` with `{'search_list': ['https://www.intel.com/content/dam/www/public/us/en/documents/datasheets/7-series-chipset-pch-datasheet.pdf'